In [1]:
import geopandas
import scipy.io
import pandas as pd
import numpy as np
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
Loading biological data ...


/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:274: UserWarning: We are currently using wgt_total from Matlab for CV, change this!
  warnings.warn("We are currently using wgt_total from Matlab for CV, change this!")


In [2]:
krig_mesh = epro_2019.get_kriging_mesh()

In [3]:
# get geopandas representation of final_biomass_table
# allows us to plot transect data
df = epro_2019.final_biomass_table
gdf = geopandas.GeoDataFrame(df, 
                             geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

# reseting the index so that we can select data based on column name
gdf = gdf.reset_index()

## Plots transect points and the full mesh 

In [4]:
# # Plot transect points 
# fmap = krig_mesh.plot_points(gdf, cmap_column='Transect', color='hex')

# # Plot full mesh points 
# fmap = krig_mesh.plot_points(krig_mesh.mesh_gdf, fmap, color='red')

# # display the folium map
# fmap

In [5]:
# transform the data using the Python version of EchoPro
# transformed_transect_gdf = krig_mesh.apply_longitude_transformation(gdf)
# transformed_mesh_gdf = krig_mesh.apply_longitude_transformation(krig_mesh.mesh_gdf, 
#                                                                 gdf_lon_name="Longitude of centroid", 
#                                                                 gdf_lat_name="Latitude of centroid")

In [6]:
# get kriging input data from Matlab version of EchoPro
krig_input_mat = scipy.io.loadmat('../2019_consolidated_files/kriging_input_points.mat')

x_data = krig_input_mat['xn'].flatten()
y_data = krig_input_mat['yn'].flatten()
field_data = krig_input_mat['var'].flatten()

x_mesh = krig_input_mat['xp'].flatten()
y_mesh = krig_input_mat['yp'].flatten()

In [7]:
from EchoPro.semivariogram import SemiVariogram as SV

In [8]:
%%time 
krig = epro_2019.get_kriging()

k_max = 10
k_min = 3
R = 0.0226287
ratio = 0.001
s_v_params = {'nugget': 0.0, 'sill': 0.95279, 'ls': 0.0075429,
              'exp_pow': 1.5, 'ls_hole_eff': 0.0}
s_v_model = SV.generalized_exp_bessel

ep_arr, eps_arr, vp_arr = krig.run_kriging(x_mesh, x_data, 
                                           y_mesh, y_data, field_data, 
                                           k_max, k_min, R, ratio, 
                                           s_v_params, s_v_model)

CPU times: user 5.1 s, sys: 3.68 s, total: 8.78 s
Wall time: 10.4 s


In [9]:
# get kriging output data from Matlab version of EchoPro
# krig_output_mat = scipy.io.loadmat('../2019_consolidated_files/kriging_output_vals.mat')
krig_output_mat = scipy.io.loadmat('../2019_consolidated_files/kriging_output_vals_m2_unity_chng_bug_fix.mat')
ep_matlab = krig_output_mat['Ep'].flatten()
eps_matlab = krig_output_mat['Eps'].flatten()
vp_matlab = krig_output_mat['Vp'].flatten()

print(np.allclose(ep_arr, ep_matlab))
print(np.allclose(eps_arr, eps_matlab, equal_nan=True))
print(np.allclose(vp_arr, vp_matlab))

True
True
True
